In [1]:
import pandas as pd
import numpy as np
import datetime
import random

### Create Data

In [2]:
# Create lists of lists, grades and number one column
location_list = ['Austin','Austin','San Antonio','Laredo','Dallas','El Paso','Chicago','Naperville','Springfield','Miami','Orlando','Austin','Austin','San Antonio','Laredo','Dallas','El Paso','Chicago','Naperville','Springfield','Miami','Orlando','Austin','Austin','San Antonio','Laredo','Dallas','El Paso','Chicago','Naperville','Springfield','Miami','Orlando']
state_list = ['TX','TX','TX','TX','TX','TX','IL','IL','IL','FL','FL','TX','TX','TX','TX','TX','TX','IL','IL','IL','FL','FL','TX','TX','TX','TX','TX','TX','IL','IL','IL','FL','FL']
name_list = ['Juan','Ellie','George','Maria','Test','Gina','Andrew','Joe','Mary','Tabelon','Wuda','Juan','Ellie','George','Maria','Test','Gina','Andrew','Joe','Mary','Tabelon','Wuda','Juan','Ellie','George','Maria','Test','Gina','Andrew','Joe','Mary','Tabelon','Wuda']
lastname_list = ['Martinez','Smith','Garcia','Logit','Account','Perdue','Lablanc','Dile','Smith','Legruti','Jenkins','Martinez','Smith','Garcia','Logit','Account','Perdue','Lablanc','Dile','Smith','Legruti','Jenkins','Martinez','Smith','Garcia','Logit','Account','Perdue','Lablanc','Dile','Smith','Legruti','Jenkins']
challenge_grades = [100,88,100,100,55,75,81,77,32,100,30,99,95,66,100,60,86,83,45,75,95,0,70,100,83,100,90,65,55,81,99,81,75]
challenge_number = [1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3]

# Create dictionary of lists
class_data = { 'location': location_list,
               'state': state_list,
               'name': name_list,
               'lastname': lastname_list,
               'challenge_grades': challenge_grades,
               'challenge_number': challenge_number
             }

# create df
df = pd.DataFrame(data=class_data)
df.sort_values(by=["name","challenge_number"]).head(5)

,location,state,name,lastname,challenge_grades,challenge_number
6,Chicago,IL,Andrew,Lablanc,81,1
17,Chicago,IL,Andrew,Lablanc,83,2
28,Chicago,IL,Andrew,Lablanc,55,3
1,Austin,TX,Ellie,Smith,88,1
12,Austin,TX,Ellie,Smith,95,2


In [3]:
# Utility function to generate random date
def generate_random_date():
    """Return random date between start_date and end_date"""
    # First class
    start_date = datetime.datetime(2021, 10, 19)
    # Current class
    end_date = datetime.datetime(2021, 11, 18)
    # Calculate days between dates
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    # Generate random number of days between those two dates
    random_number_of_days = random.randrange(days_between_dates)    
    # Use delta function to add date to start_date
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    return random_date


For each record, generate a random submission date

In [4]:
# Generate random submission date
df['submission_date'] = [generate_random_date() for n in range(len(df))]
# Create day of the week based on submission_date
df['day_of_week'] = df['submission_date'].dt.day_name()

In [5]:
df.sample(5)

,location,state,name,lastname,challenge_grades,challenge_number,submission_date,day_of_week
31,Miami,FL,Tabelon,Legruti,81,3,2021-11-13,Saturday
29,Naperville,IL,Joe,Dile,81,3,2021-11-09,Tuesday
1,Austin,TX,Ellie,Smith,88,1,2021-11-08,Monday
5,El Paso,TX,Gina,Perdue,75,1,2021-11-10,Wednesday
11,Austin,TX,Juan,Martinez,99,2,2021-10-29,Friday


In [6]:
df.dtypes

location                    object
state                       object
name                        object
lastname                    object
challenge_grades             int64
challenge_number             int64
submission_date     datetime64[ns]
day_of_week                 object
dtype: object

## Create pivot table by challenge_number 

**df.pivot reshapes dataframe**

Summarize across categorical variable(s)


### Show challenge grades for each student (name, lastname)

In [7]:
df_pivot = df.pivot(index='challenge_number', columns=['name','lastname'], values='challenge_grades')
df_pivot

name,Juan,Ellie,George,Maria,Test,Gina,Andrew,Joe,Mary,Tabelon,Wuda
lastname,Martinez,Smith,Garcia,Logit,Account,Perdue,Lablanc,Dile,Smith,Legruti,Jenkins
challenge_number,,,,,,,,,,,
1,100,88,100,100,55,75,81,77,32,100,30
2,99,95,66,100,60,86,83,45,75,95,0
3,70,100,83,100,90,65,55,81,99,81,75


!['pivot'](images/pivot_example_1.jpg)

## Create same pivot table using pivot_table

Show challenge grades for each student (name, lastname)

In [8]:
df_pivot_table = df.pivot_table(index='challenge_number', columns=['name','lastname'], values='challenge_grades', aggfunc='sum')
df_pivot_table

name,Andrew,Ellie,George,Gina,Joe,Juan,Maria,Mary,Tabelon,Test,Wuda
lastname,Lablanc,Smith,Garcia,Perdue,Dile,Martinez,Logit,Smith,Legruti,Account,Jenkins
challenge_number,,,,,,,,,,,
1,81,88,100,75,77,100,100,32,100,55,30
2,83,95,66,86,45,99,100,75,95,60,0
3,55,100,83,65,81,70,100,99,81,90,75


## Number of submissions per day of week by state


In [9]:
# Add a day type column to indicate whether day is weekend or weekday
df['day_type'] = df['day_of_week'].apply(lambda cell_value: 'Weekend' if cell_value in ['Saturday','Sunday'] else 'Weekday')

In [10]:
df_pivot_table = df.pivot_table(index=['day_type','day_of_week'], 
                                columns=['state'], values='challenge_grades', 
                                aggfunc='count', fill_value=0, margins=True, margins_name='Total')
df_pivot_table

state                 FL  IL  TX  Total
day_type day_of_week                   
Weekday  Friday        1   0   1      2
         Monday        0   1   5      6
         Thursday      0   0   2      2
         Tuesday       2   4   2      8
         Wednesday     2   2   2      6
Weekend  Saturday      1   1   6      8
         Sunday        0   1   0      1
Total                  6   9  18     33

## Average score by submission day

In [11]:
df_pivot_table = df.pivot_table(index=['day_type','day_of_week'], 
                                columns=['state'], values='challenge_grades', 
                                aggfunc='mean', fill_value=0, margins=True, margins_name='Total')
df_pivot_table

state                    FL         IL         TX      Total
day_type day_of_week                                        
Weekday  Friday       100.0   0.000000  99.000000  99.500000
         Monday         0.0  83.000000  87.600000  86.833333
         Thursday       0.0   0.000000  85.000000  85.000000
         Tuesday       52.5  71.000000  82.500000  69.250000
         Wednesday     47.5  53.500000  87.500000  62.833333
Weekend  Saturday      81.0  55.000000  80.833333  77.625000
         Sunday         0.0  99.000000   0.000000  99.000000
Total                  63.5  69.777778  85.111111  77.000000